## Matching and Subclassification

### Conditioning Strategy
- Treatment Group과 Control Group을 무작위로 배정하는 것은 현실적으로 한계가 있다.
  
- 따라서 외부 X 변수들을 통제하여 무작위와 최대한 가깝게 설계할 수 있는 방법에 대해 생각해야 된다.
  
- Conditional Independence Assumption(CIA)을 만족하며 $(Y^1,Y^0) \perp D|X$ 이고, 수식이 의미하는 바는 다음과 같다.
\begin{align}
   E\big[Y^1\mid D=1,X\big]=E\big[Y^1\mid D=0,X\big]
   \\
   E\big[Y^0\mid D=1,X\big]=E\big[Y^0\mid D=0,X\big]
\end{align}
- 조건화 전략에는 여러 방법들이 있지만, 여기서는 3가지 방법에 대해 공부해보자.
    - Subclassification
  
    - Exact Matching
  
    - Approximate Matching

#### Subclassification
- 계층별 가중치로 평균 차이에 가중치를 부여하는 방법

- 수식은 다음과 같다.
\begin{align}
\widehat{\delta}_{ATT} = \sum_{k=1}^K\Big(\overline{Y}^{1,k} - \overline{Y}^{0,k}\Big)\times \bigg( \dfrac{N^k}{N} \bigg )
\end{align}
- $K$ = 각 범주의 수 (만약 2개의 연령 범주와 2 개의 성별 범주가 있다면, 4가지의 수가 존재, $K = 4$)
  
- ${N^k}$ = 각 범주에 따른 데이터 수
  
- 따라서 해당 방법에는 변수와 범주의 수가 많을 경우, $K$의 수가 급격히 커지기 때문에 차원의 저주라는 문제점이 있다.

#### Exact Matching
- Subclassification 방법에는 차원의 저주라는 문제점이 있어서 다른 방법을 고안해야 된다.
  
- Exact Matching: 누락된 각 관측값(unit)의 잠재적 결과가 있을 때, Treatment unit과 Control unit X의 동일한 값을 매칭하여 누락된 값을 다른 그룹의 유사한 값으로 대체하는 방법이다.

##### Exact Matching Example
- 연수 프로그램이 수입에 미치는 영향을 추정해보자
  
    - trainees: 연수 프로그램 유무
  
    - earnings: 수입

In [6]:
import pandas as pd
trainee = pd.read_csv("C:\\Users\\이찬영\\Desktop\\trainees.csv") # 데이터 불러오기

- 나이 (X)가 Treatment Group 과 Control Group 간 서로 다르기 때문에 단순 차이를 비교하기 어렵다.

In [29]:
print('매칭을 활용하지 않은 단순 평균 차이:', trainee.query("trainees==1")["earnings"].mean() - trainee.query("trainees==0")["earnings"].mean())
print("연수생 O 나이 평균:",trainee.query("trainees==1")["age"].mean())
print("연수생 X 나이 평균:",trainee.query("trainees==0")["age"].mean())

매칭을 활용하지 않은 단순 평균 차이: -4297.49373433584
연수생 O 나이 평균: 28.473684210526315
연수생 X 나이 평균: 33.0


In [23]:
# make dataset where no one has the same age
unique_on_age = (trainee
                 .query("trainees==0")
                 .drop_duplicates("age"))

matches = (trainee
           .query("trainees==1")
           .merge(unique_on_age, on="age", how="left", suffixes=("_t_1", "_t_0"))
           .assign(t1_minuts_t0 = lambda d: d["earnings_t_1"] - d["earnings_t_0"]))

matches.head(7)

,unit_t_1,trainees_t_1,age,earnings_t_1,unit_t_0,trainees_t_0,earnings_t_0,t1_minuts_t0
0,1,1,28,17700,27,0,8800,8900
1,2,1,34,10200,34,0,24200,-14000
2,3,1,29,14400,37,0,6200,8200
3,4,1,25,20800,35,0,23300,-2500
4,5,1,29,6100,37,0,6200,-100
5,6,1,23,28600,40,0,9500,19100
6,7,1,33,21900,29,0,15500,6400


- Result

In [28]:
print('매칭을 활용하지 않은 단순 평균 차이:', trainee.query("trainees==1")["earnings"].mean() - trainee.query("trainees==0")["earnings"].mean())
print("매칭을 활용한 평균 차이:",matches["t1_minuts_t0"].mean())

매칭을 활용하지 않은 단순 평균 차이: -4297.49373433584
매칭을 활용한 평균 차이: 2457.8947368421054


#### Approximate Matching
- 현실적으로 정확히 같은 다른 단위 값을 찾는 것은 어려운 문제이기 때문에 대략적으로 일치한 값과 매칭하여 추정하는 방법을 고안해야 된다.
  
- 한 가지 방법은 X의 거리를 측정하여 가장 가까운 값에 대체시키는 방법이다.
  
    - Nearest neighbor covariate matching
  
    - $||X_i-X_j||=\sqrt{ (X_i-X_j)'\widehat{\Sigma}_X^{-1}(X_i - X_j) }$
  
    - 거리의 개념을 활용한 수학적 공식은 다음과 같다.
\begin{align}
\widehat{\delta}_{ATE} = \dfrac{1}{N} \sum_{i=1}^N (2D_i - 1) \bigg [ Y_i - \bigg ( \dfrac{1}{M} \sum_{m=1}^M Y_{j_m(i)} \bigg ) \bigg ]
\end{align}
- $Y_{j_m(i)}$ = $X$에 대해 $i$단위와 가장 가까운 $j$단위의 $Y$ 결과 값($m$ 개)에 대한 평균 값
- $2D_i - 1$ = Treatment unit과 Control unit의 잠재적 결과가 unit에 따라 다르므로 Trick 적용

##### Nearest neighbor covariate matching Example
- 약물의 효과에 따라 환자 회복일 수에 차이가 있는지 살펴보자.

In [30]:
med = pd.read_csv("C:\\Users\\이찬영\\Desktop\\medicine_impact_recovery.csv")
med.head()

,sex,age,severity,medication,recovery
0,0,35.049134,0.887658,1,31
1,1,41.580323,0.899784,1,49
2,1,28.127491,0.486349,0,38
3,1,36.375033,0.323091,0,35
4,0,25.091717,0.209006,0,15


- 각 변수들의 스케일을 먼저 조정한 후, 관측치 사이의 거리를 계산해야된다.
  
    - 스케일 조정하는 이유: 한 변수의 값이 극도로 클 경우, 계산하는데 영향을 미치기 때문

In [39]:
# scale features
X = ["severity", "age", "sex"]
y = "recovery"

med = med.assign(**{f: (med[f] - med[f].mean())/med[f].std() for f in X})
med.head()

,sex,age,severity,medication,recovery
0,-0.996980,0.280787,1.459800,1,31
1,1.002979,0.865375,1.502164,1,49
2,1.002979,-0.338749,0.057796,0,38
3,1.002979,0.399465,-0.512557,0,35
4,-0.996980,-0.610473,-0.911125,0,15


- KNN 모델을 활용한 매칭 (k-nearest-neighbor)

In [40]:
from sklearn.neighbors import KNeighborsRegressor

treated = med.query("medication==1")
untreated = med.query("medication==0")

mt0 = KNeighborsRegressor(n_neighbors=1).fit(untreated[X], untreated[y])
mt1 = KNeighborsRegressor(n_neighbors=1).fit(treated[X], treated[y])

predicted = pd.concat([
    # find matches for the treated looking at the untreated knn model
    treated.assign(match=mt0.predict(treated[X])),
    
    # find matches for the untreated looking at the treated knn model
    untreated.assign(match=mt1.predict(untreated[X]))
])

predicted.head()

,sex,age,severity,medication,recovery,match
0,-0.996980,0.280787,1.459800,1,31,39.0
1,1.002979,0.865375,1.502164,1,49,52.0
7,-0.996980,1.495134,1.268540,1,38,46.0
10,1.002979,-0.106534,0.545911,1,34,45.0
16,-0.996980,0.043034,1.428732,1,30,39.0


- Result

In [43]:
print("단순 평균 차이:",med.query("medication==1")["recovery"].mean() - med.query("medication==0")["recovery"].mean())
print("KNN을 활용한 평균 차이:",np.mean((2*predicted["medication"] - 1)*(predicted["recovery"] - predicted["match"])))

단순 평균 차이: 16.895799546498726
KNN을 활용한 평균 차이: -0.9954


### Bias Correction
- 하지만, Matching을 통한 인과추론에서 편향 문제가 발생한다.

- 따라서 편향을 보정한 수학적 공식은 다음과 같다.
\begin{align}
   \widehat{\delta}_{ATT}^{BC} = \dfrac{1}{N_T} \sum_{D_i=1} \bigg [ (Y_i - Y_{j(i)}) - \Big(\widehat{\mu}^0(X_i) - \widehat{\mu}^0(X_{j(i)})\Big) \bigg ]
\end{align}

- $Proof)$ 
\begin{align}
   Y_i = \mu^{D_i}(X_i) + \varepsilon_i
\end{align}

\begin{align}
   \mu^0(x) & = E\big[Y\mid X=x, D=0\big] = E\big[Y^0\mid X=x\big] \\
   \mu^1(x) & = E\big[Y\mid X=x, D=1\big] = E\big[Y^1\mid X=x\big]
\end{align}

- 위 식은 $X$를 활용한 각 $Y^1$, $Y^0$의 값을 회귀분석의 식으로 표현하였다.
  
  - Control Group: $Y^1$ (Counterfactual)
  
  - Treatment Group: $Y^0$ (Counterfactual)
- 따라서 $\widehat{\delta}_{ATT} = \dfrac{1}{N_T} \sum_{D_i=1}(Y_i - Y_{j(i)})$은 $\dfrac{1}{N_T} \sum_{D_i=1} \big(\mu^1(X_i) + \varepsilon_i\big) - \big(\mu^0(X_{j(i)}\big) + \varepsilon_{j(i)})$ 와 같다.
\begin{align}
   \widehat{\delta}_{ATT}
     & =\dfrac{1}{N_T} \sum_{D_i=1} \big(\mu^1(X_i) + \varepsilon_i\big) - \big(\mu^0(X_{j(i)}\big) + \varepsilon_{j(i)})\big)                             
   \\
     & =\dfrac{1}{N_T} \sum_{D_i=1} \big(\mu^1(X_i) - \mu^0(X_{j(i)})\big) + \dfrac{1}{N_T} \sum_{D_i=1} \big(\varepsilon_i - \varepsilon_{j(i)}\big)
\end{align}

\begin{align}
\widehat{\delta}_{ATT} - \delta_{ATT} = \dfrac{1}{N_T} \sum_{D_i=1} \left(\mu^1(X_i) - \mu^0(X_{j(i)})\right) - \delta_{ATT}
   + \dfrac{1}{N_T} \sum_{D_i=1}\big(\varepsilon_i - \varepsilon_{j(i)}\big)
\end{align}

\begin{align}
   \widehat{\delta}_{ATT} - \delta_{ATT}
     & = \dfrac{1}{N_T} \sum_{D_i=1} \left( \mu^1(X_i) - {\mu^0(X_i)}\right) - \delta_{ATT}
   \\
     & + \dfrac{1}{N_T} \sum_{D_i=1} (\varepsilon_i - \varepsilon_{j(i)})                   
   \\
     & + \dfrac{1}{N_T} \sum_{D_i=1} \left( {\mu^0(X_i)} - \mu^0(X_{j(i)}) \right).         
\end{align}
  - 중심극한정리에 의해 다음 식이 성립된다.
  \begin{align}
  E\Big[ \sqrt{N_T} (\widehat{\delta}_{ATT} - \delta_{ATT})\Big] = E\Big[ \sqrt{N_T}(\mu^0(X_i)-\mu^0(X_{j(i)}) )\mid D=1\Big]
  \end{align}
- $\mu^0(X_i)$ 는 처치된 unit $i$가 처치되지 않았더라면 계산될 $Y$ 값이며, $\mu^0(X_{j(i)})$는 실제로 볼 수 있는 사실적 결과이다.

  - 공변량의 크기가 커질수록 $X_i$와 $X_{j(i)}$의 차이는 점점 느리게 수렴하는 반면, $\sqrt{N_T}$는 그 차이에 비해 더 빠르게 증가하기 때문에 최종적으로 0에 수렴하지 못하여 편향이 발생하게 된다.

##### Bias Correction Matching Example
- 약물의 효과를 편향 보정하여 다시 추정해보자.

In [44]:
from sklearn.linear_model import LinearRegression

# fit the linear regression model to estimate mu_0(x)
ols0 = LinearRegression().fit(untreated[X], untreated[y])
ols1 = LinearRegression().fit(treated[X], treated[y])

# find the units that match to the treated
treated_match_index = mt0.kneighbors(treated[X], n_neighbors=1)[1].ravel()

# find the units that match to the untreatd
untreated_match_index = mt1.kneighbors(untreated[X], n_neighbors=1)[1].ravel()

predicted = pd.concat([
    (treated
     # find the Y match on the other group
     .assign(match=mt0.predict(treated[X])) 
     
     # build the bias correction term
     .assign(bias_correct=ols0.predict(treated[X]) - ols0.predict(untreated.iloc[treated_match_index][X]))),
    (untreated
     .assign(match=mt1.predict(untreated[X]))
     .assign(bias_correct=ols1.predict(untreated[X]) - ols1.predict(treated.iloc[untreated_match_index][X])))
])

predicted.head()

,sex,age,severity,medication,recovery,match,bias_correct
0,-0.996980,0.280787,1.459800,1,31,39.0,4.404034
1,1.002979,0.865375,1.502164,1,49,52.0,12.915348
7,-0.996980,1.495134,1.268540,1,38,46.0,1.871428
10,1.002979,-0.106534,0.545911,1,34,45.0,-0.496970
16,-0.996980,0.043034,1.428732,1,30,39.0,2.610159


- Result

In [46]:
print("단순 평균 차이:",med.query("medication==1")["recovery"].mean() - med.query("medication==0")["recovery"].mean())
print("KNN을 활용한 평균 차이:",np.mean((2*predicted["medication"] - 1)*(predicted["recovery"] - predicted["match"])))
print("KNN을 활용한 후, 편향 보정한 평균 차이:",np.mean((2*predicted["medication"] - 1)*((predicted["recovery"] - predicted["match"])-predicted["bias_correct"])))

단순 평균 차이: 16.895799546498726
KNN을 활용한 평균 차이: -0.9954
KNN을 활용한 후, 편향 보정한 평균 차이: -7.362660906141416
